# Utilize fine-tune model

There are three main approaches to this:

1. Download a model from HuggingFace directly, in my case that would be: didierlopes/phi-3-mini-4k-instruct-ft-on-didier-blog

2. Load a base model from HuggingFace (e.g. microsoft/Phi-3-mini-4k-instruct) with the LoRA adapters from the fine-tuning which exist locally on the machine - using MLX

3. Load a local fused model (base model + LoRA adapters) - using MLX

### 1. Run model off HuggingFace

The assumption is that you have pushed your fine-tuned model to HuggingFace.

```
pip install torch torchvision torchaudio
pip install transformers
pip install 'accelerate>=0.26.0'
```

In [1]:
# Import necessary libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Set up the model and tokenizer
model_name = "didierlopes/phi-3-mini-4k-instruct-ft-on-didier-blog"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Define a function to generate text
def generate_text(prompt, max_length=100):
    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.9,
            do_sample=True
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

/Users/didierlopes/miniconda3/envs/bsky/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.76s/it]


In [3]:
from transformers.utils import TRANSFORMERS_CACHE

# Check if the model is stored in cache
print(f"Models are cached in: {TRANSFORMERS_CACHE}")

Models are cached in: /Users/didierlopes/.cache/huggingface/hub


In [4]:
# Provide a sample prompt and generate output
generated_output = generate_text(
    "What is the OpenBB workspace? How is it different from others?"
)

print(generated_output)

What is the OpenBB workspace? How is it different from others?
The OpenBB workspace is a collaborative platform for finance professionals, powered by our AI engine, designed to streamline workflows and improve productivity. It's unique in its ability to integrate with multiple financial tools and data sources, allowing users to create custom workflows and pipelines. Our workspace also offers advanced features such as natural language processing and machine learning to enhance decision-making


### 2. Load base model of HuggingFace with local LoRA adapters (using MLX)

```
conda install -c conda-forge mlx
CONDA_SUBDIR=osx-arm64 conda create -n bsky python=3.11
conda activate bsky
conda config --env --set subdir osx-arm64
```

In [5]:
from mlx_lm import load, generate

model_path = "microsoft/Phi-3-mini-4k-instruct"

model_lora, tokenizer_lora = load(
    model_path,
    adapter_path="../../fine-tune-llm/adapters"
)

output = generate(
    model_lora,
    tokenizer_lora,
    "What is the OpenBB workspace? How is it different from others?",
    max_tokens=200
)

print(output)

Fetching 13 files: 100%|██████████| 13/13 [00:00<00:00, 203455.04it/s]



The OpenBB workspace is a collaborative platform for financial analysts to share insights, data, and tools. It's different from others because it's built on open-source technology, allowing users to customize and extend the platform to fit their needs. This openness fosters a community-driven approach to financial analysis.


### 3. Load fused model locally (using MLX)

```
CONDA_SUBDIR=osx-arm64 conda create -n fine-tune-llm python=3.11
conda activate fine-tune-llm
conda config --env --set subdir osx-arm64
```

In [6]:
from mlx_lm import load, generate

fused_model, fused_tokenizer = load("../../fine-tune-llm/lora_fused_model")

output = generate(
    fused_model,
    fused_tokenizer,
    "What is the OpenBB workspace? How is it different from others?",
    max_tokens=200
)

print(output)


The OpenBB workspace is a collaborative platform that allows users to share and discuss financial data, research, and insights. It's different from others in that it's built on top of the OpenBB Terminal, providing a seamless integration between the terminal and the workspace. This allows users to easily access and share data, making it a powerful tool for financial analysis and decision-making.
